In [47]:
import optuna
from optuna.samplers import TPESampler
import plotly
import plotly.express as px
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, precision_recall_curve, f1_score, auc
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [48]:
data = pd.read_csv('cleanedData.csv')
print(data.shape)
data.head()

(2930, 76)


,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,...,3Ssn Porch,Screen Porch,Fence,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,HasFence
0,1,526301100,20,RL,141.0,31770,1,IR1,Lvl,AllPub,...,0,0,NoFence,0,5,2010,WD,Normal,215000,0
1,2,526350040,20,RH,80.0,11622,1,Reg,Lvl,AllPub,...,0,120,MnPrv,0,6,2010,WD,Normal,105000,1
2,3,526351010,20,RL,81.0,14267,1,IR1,Lvl,AllPub,...,0,0,NoFence,12500,6,2010,WD,Normal,172000,0
3,4,526353030,20,RL,93.0,11160,1,Reg,Lvl,AllPub,...,0,0,NoFence,0,4,2010,WD,Normal,244000,0
4,5,527105010,60,RL,74.0,13830,1,IR1,Lvl,AllPub,...,0,0,MnPrv,0,3,2010,WD,Normal,189900,1


In [49]:
X, y = data.drop("SalePrice", axis = 'columns'), data['SalePrice']
X.head()


,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,...,Enclosed Porch,3Ssn Porch,Screen Porch,Fence,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,HasFence
0,1,526301100,20,RL,141.0,31770,1,IR1,Lvl,AllPub,...,0,0,0,NoFence,0,5,2010,WD,Normal,0
1,2,526350040,20,RH,80.0,11622,1,Reg,Lvl,AllPub,...,0,0,120,MnPrv,0,6,2010,WD,Normal,1
2,3,526351010,20,RL,81.0,14267,1,IR1,Lvl,AllPub,...,0,0,0,NoFence,12500,6,2010,WD,Normal,0
3,4,526353030,20,RL,93.0,11160,1,Reg,Lvl,AllPub,...,0,0,0,NoFence,0,4,2010,WD,Normal,0
4,5,527105010,60,RL,74.0,13830,1,IR1,Lvl,AllPub,...,0,0,0,MnPrv,0,3,2010,WD,Normal,1


In [50]:
total = 0 
bins = 0
lims = [] #limita superioara = valori pana la ....
frequency = [] #numarul de valori pana la limita superioara corespunzatoare
sorted_values = data['SalePrice'].value_counts().sort_index()
for i, j in sorted_values.items():
    if total + j > 100:
        bins += 1 
        lims.append(str(i))
        frequency.append(total)
        total = j
    else:
        total = total + j
print(bins)
print(lims)
print(frequency)

30
['81500', '95000', '106000', '113000', '119000', '124000', '128000', '131000', '135000', '138000', '141500', '145000', '148800', '154000', '159000', '164990', '170440', '176000', '181000', '187500', '194500', '204000', '214000', '225000', '237000', '252678', '272000', '300000', '338931', '438780']
[100, 98, 99, 98, 90, 98, 99, 97, 76, 92, 100, 77, 99, 99, 100, 100, 100, 89, 96, 98, 100, 97, 98, 96, 98, 100, 100, 98, 100, 100]


In [51]:
lims_numeric = sorted([int(x) for x in lims])
lims_numeric.append(int(data['SalePrice'].max() + 1))
y_classEq = pd.cut(data['SalePrice'], bins=[0]+lims_numeric, labels=False, include_lowest=True).astype(int)


In [52]:
X_numeric = pd.get_dummies(X) 
X_train, X_test, y_train, y_test = train_test_split(
    X_numeric, y_classEq, test_size=0.1, random_state=42
)

In [53]:
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0)
    }

    model = XGBClassifier(
        **params,
        eval_metric='mlogloss',
        objective='multi:softprob',
        num_class=len(np.unique(y_classEq))
    )

    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)

    f1 = f1_score(y_test, y_pred, average='macro')
    return f1

In [54]:
study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=30)


[I 2025-11-04 17:45:50,212] A new study created in memory with name: no-name-2fca3685-ef8b-4b6b-884b-88f8a7ad7975
[I 2025-11-04 17:46:04,306] Trial 0 finished with value: 0.21040374788181943 and parameters: {'max_depth': 7, 'learning_rate': 0.18944801123441593, 'n_estimators': 790, 'subsample': 0.5771644297935697, 'colsample_bytree': 0.7328726563070624, 'gamma': 0.9972766579553632, 'reg_alpha': 0.12618337642495714, 'reg_lambda': 0.814219882117576}. Best is trial 0 with value: 0.21040374788181943.
[I 2025-11-04 17:46:14,045] Trial 1 finished with value: 0.19386869208604324 and parameters: {'max_depth': 10, 'learning_rate': 0.2015273411407249, 'n_estimators': 756, 'subsample': 0.630520868395312, 'colsample_bytree': 0.5261551089453336, 'gamma': 3.7899396359706827, 'reg_alpha': 0.7551875137968467, 'reg_lambda': 0.7916459674235017}. Best is trial 0 with value: 0.21040374788181943.
[I 2025-11-04 17:46:20,958] Trial 2 finished with value: 0.17208412842327883 and parameters: {'max_depth': 6, '

In [55]:
best_trial = study.best_trial
print("Best Macro F1:", best_trial.value)
print("Best hyperparameters:", best_trial.params)


Best Macro F1: 0.24682643128738205
Best hyperparameters: {'max_depth': 9, 'learning_rate': 0.0662646532048138, 'n_estimators': 294, 'subsample': 0.6861000350050542, 'colsample_bytree': 0.7992215395837252, 'gamma': 2.0546655533284106, 'reg_alpha': 0.634954704896851, 'reg_lambda': 0.6376172437412999}


In [56]:
best_params = best_trial.params
final_model = XGBClassifier(
    **best_params,
    eval_metric='mlogloss',
    objective='multi:softprob',
    num_class=len(np.unique(y_classEq))
)

final_model.fit(X_train, y_train)


,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7992215395837252
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'mlogloss'
